## Importing libraries

In [1]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

## Functions

In [2]:
# Function to create the input function
def makeInputFunction(data, labels, num_epochs=10, shuffle=True, batch_size=32):
    def inputFunction():
        ds = tf.data.Dataset.from_tensor_slices((dict(data), labels))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return inputFunction

# Function to split data into batches of a given size
def splitIntoBatches(df, batch_size=100000):
    numberOfRows = len(df.index)
    intervals = np.arange(0, numberOfRows + 1, batch_size)
    intervals[-1] = numberOfRows

    batchesX = []
    batchesY = []

    for i in range(len(intervals) - 1):
        batchX = df.iloc[intervals[i]:intervals[i + 1]][features].copy()
        batchY = df.iloc[intervals[i]:intervals[i + 1]]['good_move'].copy()
        batchesX.append(batchX)
        batchesY.append(batchY)

    return batchesX, batchesY

## Data

In [3]:
path = "Data/CSV_FISCHER"

files = glob.glob(path + "/*.csv")

# Creating one single dataframe by concatenating all the files
array = []
for i in files:
    df = pd.read_csv(i, index_col=None, header=0)
    array.append(df)

train = pd.concat(array, axis=0, ignore_index=True)
train = shuffle(train)
features = list(train.iloc[:, 0:192].columns)
x = train[features]
y = train["good_move"]

# Specifying the feature columns
categoricalColumns = list(x.iloc[:, 0:63].columns)
numericalColumns = list(x.iloc[:, 64:192].columns)
featureColumns = []

for featureName in categoricalColumns:
    vocabulary = x[featureName].unique()
    featureColumns.append(tf.feature_column.categorical_column_with_vocabulary_list(featureName, vocabulary))

for featureName in numericalColumns:
    featureColumns.append(tf.feature_column.numeric_column(featureName, dtype=tf.float32))

# Splitting the dataset into batches
batchesX, batchesY = splitIntoBatches(train)

## Model

In [4]:
estimator = tf.estimator.LinearClassifier(feature_columns=featureColumns)

# Training the model on all the input functions
for dfX, dfY in zip(batchesX, batchesY):
    trainInputFunction = makeInputFunction(dfX, dfY)
    estimator.train(trainInputFunction)

# Saving the model
servingInputReceiverFunction = tf.estimator.export.build_parsing_serving_input_receiver_fn(tf.feature_column.make_parse_example_spec(featureColumns))
pathModel = estimator.export_saved_model(r"C:\Users\giaco\OneDrive\Desktop\Informatica\Master\Codes, Projects and Researches\AI Chess\Model", servingInputReceiverFunction)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\giaco\\AppData\\Local\\Temp\\tmp3tdib8hm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_v

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\giaco\AppData\Local\Temp\tmp3tdib8hm\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 18.8754
INFO:tensorflow:loss = 0.1494098, step = 100 (5.298 sec)
INFO:tensorflow:global_step/sec: 116.316
INFO:tensorflow:loss = 0.27841258, step = 200 (0.860 sec)
INFO:tensorflow:global_step/sec: 112.184
INFO:tensorflow:loss = 0.14761445, step = 300 (0.891 sec)
INFO:tensorflow:global_step/sec: 110.339
INFO:tensorflow:loss = 0.3933125, step = 400 (

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmp3tdib8hm\model.ckpt-31250
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 31250...
INFO:tensorflow:Saving checkpoints for 31250 into C:\Users\giaco\AppData\Local\Temp\tmp3tdib8hm\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 31250...
INFO:tensorflow:loss = 0.03814295, step = 31250
INFO:tensorflow:global_step/sec: 17.9394
INFO:tensorflow:loss = 0.026911017, step = 31350 (5.574 sec)
INFO:tensorflow:global_step/sec: 104.423
INFO:tensorflow:loss = 0.1348266, step = 31450 (0.958 sec)
INFO:tensorflow:global_step/sec: 110.506
INFO:tensorflow:loss = 0.1816477, step = 31550 (0.905 sec)
INFO:te

...

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmp3tdib8hm\model.ckpt-156250
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 156250...
INFO:tensorflow:Saving checkpoints for 156250 into C:\Users\giaco\AppData\Local\Temp\tmp3tdib8hm\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 156250...
INFO:tensorflow:loss = 0.03093446, step = 156250
INFO:tensorflow:global_step/sec: 16.3615
INFO:tensorflow:loss = 0.03634181, step = 156350 (6.128 sec)
INFO:tensorflow:global_step/sec: 95.5225
INFO:tensorflow:loss = 0.027006054, step = 156450 (1.031 sec)
INFO:tensorflow:global_step/sec: 95.5223
INFO:tensorflow:loss = 0.034661878, step = 156550 (1.047 sec)
INFO:tensorflow:global_step/sec: 92.7131
INFO:tensorflow:loss = 0.

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmp3tdib8hm\model.ckpt-187500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 187500...
INFO:tensorflow:Saving checkpoints for 187500 into C:\Users\giaco\AppData\Local\Temp\tmp3tdib8hm\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 187500...
INFO:tensorflow:loss = 0.14464015, step = 187500
INFO:tensorflow:global_step/sec: 16.4261
INFO:tensorflow:loss = 0.21736524, step = 187600 (6.090 sec)
INFO:tensorflow:global_step/sec: 94.9097
INFO:tensorflow:loss = 0.36328232, step = 187700 (1.052 sec)
INFO:tensorflow:global_step/sec: 92.5496
INFO:tensorflow:loss = 0.033053976, step = 187800 (1.081 sec)
INFO:tensorflow:global_step/sec: 93.5261
INFO:tensorflow:loss = 0.1

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmp3tdib8hm\model.ckpt-218750
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 218750...
INFO:tensorflow:Saving checkpoints for 218750 into C:\Users\giaco\AppData\Local\Temp\tmp3tdib8hm\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 218750...
INFO:tensorflow:loss = 0.2173784, step = 218750
INFO:tensorflow:global_step/sec: 15.4233
INFO:tensorflow:loss = 0.03559672, step = 218850 (6.484 sec)
INFO:tensorflow:global_step/sec: 90.0744
INFO:tensorflow:loss = 0.12863705, step = 218950 (1.110 sec)
INFO:tensorflow:global_step/sec: 90.7745
INFO:tensorflow:loss = 0.027266884, step = 219050 (1.102 sec)
INFO:tensorflow:global_step/sec: 91.0186
INFO:tensorflow:loss = 0.22